In [1]:
import numpy as np
from IPython.display import Image
import pdb
#
# [위치] F:\temp
# [venv] F:\venv\bento_yolo310\Scripts
# 동적계획법 이해: https://www.youtube.com/watch?v=0bqfTzpWySY
# 

In [2]:
# Image('dp요약.jpg')
# 만일 각 단계에서 이전 단계로 연결되는 경로의 최적값을 미리 계산하여 저장해 놓지 않으면,
# 탐색이 발생할 때 마다 후단의 모든 단계, 모든 경로를 재 탐색하여 다시 계산해야 한다.
# 이것은 과도하게 반복되는 계산을 발생시킴. DP는 이러한 반복연산을 LUT에 저장함에 의해 
# 제거 할 수 있음.
# LUT계산에는 현단계, 현노드에서 이전 단계로의 연결 중 최적연결을 선택(u*)하고, 이 연결과
# 그 때의 cost를 저장한다. 

In [48]:
class DP_MPC:
    x_min, x_max = -2, 2  # state range
    u_min, u_max = -1, 1  # control range
    num_grid = 20  # x, u의 격자 수
    horizon = 5
    
    def __init__(self, model=None):
        self.model = model
        self.initialize()

    def initialize(self):
        self.cost_to_go = np.inf * np.ones((self.num_grid, self.horizon+1))
        self.cost_to_go[:, -1] = 0  # 종단 비용 초기화
        #print(self.cost_to_go)
        
        # 최적 제어 입력 저장
        self.u_optimal = np.zeros((self.num_grid, self.horizon))
        self.u_step = np.zeros((self.num_grid, self.horizon))

        self.x_grid = np.linspace(self.x_min, self.x_max, self.num_grid)
        self.u_grid = np.linspace(self.u_min, self.u_max, self.num_grid)   
        #print(f"x:{self.x_grid}")
        #print(f"u:{self.u_grid}")

    def optimize(self, x0=0, control_ref=0.5):
        # DP를 이용한 비용 함수 계산
        for k in range(self.horizon-1, -1, -1):   
            # 각 단계마다 가능한 모든 x, u값을 순회함
            for ii, x in enumerate(self.x_grid):   
                # 각 상태별로 최적의 u값 선택
                for jj, u in enumerate(self.u_grid):  # 주어진 x에 대해 u의 가능한 모든 값 중 cost가 최소가 되는 u를 선택  
                    x_next = x + u  # DNN: x_t+1=f(x_t,u_t)
                    if x_next < self.x_grid[0] or x_next > self.x_grid[-1]:
                        continue
                    cost = (x-control_ref)**2 + u**2
                    idx_next = np.argmin(np.abs(self.x_grid - x_next))  # 예측된 값과 상태리스트 중 제일 가까운 값을 주는 인덱스
                    # cost_to_go[.,kk]: kk단계에서 종단까지 최적 비용 합.
                    # J(x_t)=min_u_t[c(x_t,u_t)+J(x_t+1)]: 현재 상태에서 미래에 도달할 목표까지 발생할 예상비용의 합
                    total_cost = cost + self.cost_to_go[idx_next, k+1]
                    
                    # 현재의 ii(상태값), k(단계)에 대해 가능한 모든 u를 순회하여, 
                    # 최소 비용을 주는 u값과 그 때의 cost를 LUT의 (ii,k)위치에 저장
                    if total_cost < self.cost_to_go[ii, k]:
                        self.cost_to_go[ii, k] = total_cost
                        self.u_optimal[ii, k] = u
                        self.u_step[ii, k] = jj
        
                #print(f"step:{k}, state:{ii}, x:{x:.4f}, cost:{self.cost_to_go[ii,k]:.4f} ({self.u_optimal[ii, k]:.4f})")
                #pdb.set_trace()  # c누르면서 테스트
                
            #print("----------------------")    

        return self.u_control(x0)

    def u_control(self, x):
        u_seq = []
        for k in range(self.horizon):
            idx = np.argmin(np.abs(self.x_grid - x))
            u = self.u_optimal[idx, k]
            u_seq.append(u)
            x += u
            print(f"idx:{idx}, prev x:{x-u:.4f}, x:{x:.4f}, \
                u(indx):({self.u_grid[int(self.u_step[idx,k])]:.4f},{int(self.u_step[idx,k])}), \
                cost:{self.cost_to_go[idx,k]:.4f}")        


dp_mpc = DP_MPC()

In [49]:
u_optimal = dp_mpc.optimize()

idx:10, prev x:0.0000, x:0.1579,                 u(indx):(0.1579,11),                 cost:0.2500
idx:10, prev x:0.1579, x:0.3158,                 u(indx):(0.1579,11),                 cost:0.2465
idx:11, prev x:0.3158, x:0.4737,                 u(indx):(0.1579,11),                 cost:0.0658
idx:12, prev x:0.4737, x:0.5263,                 u(indx):(0.0526,10),                 cost:0.0069
idx:12, prev x:0.5263, x:0.5789,                 u(indx):(0.0526,10),                 cost:0.0035


In [37]:
## 시스템 매개변수
# 제어입력: -1 ~ +1, horizon: 5 ~ 10, 
# 초기상태: 0, 목표상태: 0.5
# 상태값범위: -2 ~ +2
##
N = 10  # 예측 구간
Q = 1  # 상태 가중치
R = 1  # 제어 입력 가중치
x0 = 0 #0  # 초기 상태
u_max = 1  # 제어 입력 범위
u_min = -1  # 제어 입력 범위
num_grid = 30  # 상태 및 제어 입력 격자 수

control_ref = 0.5  ###### 목표 상태값


# 상태와 제어 입력 공간 격자화
x_grid = np.linspace(-2, 2, num_grid)
u_grid = np.linspace(u_min, u_max, num_grid)
print(f"x:{x_grid}")
print(f"u:{u_grid}")

# 비용 함수 초기화
cost_to_go = np.inf * np.ones((num_grid, N+1))
cost_to_go[:, -1] = 0  # 종단 비용 초기화
print(cost_to_go[:,:])

# 최적 제어 입력 저장
u_optimal = np.zeros((num_grid, N))
u_step = np.zeros((num_grid, N))

# DP를 이용한 비용 함수 계산
for k in range(N-1, -1, -1):  # 단계순서는 뒤에서 앞으로
    # 각 단계마다 가능한 모든 x, u값을 순회함
    for ii, x in enumerate(x_grid):  # x값이 바뀜에 따라 아래 best의 u값이 선택되어도 cost값은 달라짐. 이 cost를 ii위치에 저장
        # 각 상태별로 최적의 u값 선택
        for jj, u in enumerate(u_grid):  # 주어진 x에 대해 u의 가능한 모든 값 중 cost가 최소가 되는 u를 선택하고, 그 때 cost값도 저장 
            x_next = x + u  # DNN: x_t+1=f(x_t,u_t)
            if x_next < x_grid[0] or x_next > x_grid[-1]:
                continue
            cost = (x-control_ref)**2 + u**2
            idx_next = np.argmin(np.abs(x_grid - x_next))  # 예측된 상태와 가능한 상태값 중 제일 가까운 값을 주는 인덱스
            # cost(k단계비용) + cost_to_go(k+1단계에서 (u로 결정된) 어떤 x에서의 cost)
            # cost_to_go[ii,kk]: 상태값 ii에서 k단계에서 종단까지 최적 비용의 합.
            # J(x_t)=min_u_t[c(x_t,u_t)+J(x_t+1)]: 현재 상태에서 미래에 도달할 목표까지 발생할 예상비용의 합
            total_cost = cost + cost_to_go[idx_next, k+1]
            
            # 현재의 ii(상태값), k(단계)에 대해 가능한 모든 u를 순회하여, 
            # 최소 비용을 주는 u값과 그 때의 cost를 LUT의 (ii,k)위치에 저장
            if total_cost < cost_to_go[ii, k]:
                cost_to_go[ii, k] = total_cost
                u_optimal[ii, k] = u
                u_step[ii, k] = jj
                #print(k,ii,jj,idx_next, x,u,cost,cost_to_go[idx_next, k+1],total_cost)

        print(f"step:{k}, state:{ii}, x:{x:.4f}, cost:{cost_to_go[ii,k]:.4f} ({u_optimal[ii, k]:.4f}, {int(u_step[ii, k])})")
        #pdb.set_trace()  # c누르면서 테스트
        
    print("----------------------")
                

# 초기 상태에 대한 최적 제어 입력 시퀀스 생성
x = x0
u_seq = []
for k in range(N):
    idx = np.argmin(np.abs(x_grid - x))
    u = u_optimal[idx, k]
    u_seq.append(u)
    x += u
    print(f"idx:{idx}, prev x:{x-u:.4f}, x:{x:.4f}, \
        u(indx):({u_grid[int(u_step[idx,k])]:.4f},{int(u_step[idx,k])}), \
        cost:{cost_to_go[idx,k]:.4f}")


# 결과 출력
print("Optimal control sequence:")
print(u_seq)

x:[-2.         -1.86206897 -1.72413793 -1.5862069  -1.44827586 -1.31034483
 -1.17241379 -1.03448276 -0.89655172 -0.75862069 -0.62068966 -0.48275862
 -0.34482759 -0.20689655 -0.06896552  0.06896552  0.20689655  0.34482759
  0.48275862  0.62068966  0.75862069  0.89655172  1.03448276  1.17241379
  1.31034483  1.44827586  1.5862069   1.72413793  1.86206897  2.        ]
u:[-1.         -0.93103448 -0.86206897 -0.79310345 -0.72413793 -0.65517241
 -0.5862069  -0.51724138 -0.44827586 -0.37931034 -0.31034483 -0.24137931
 -0.17241379 -0.10344828 -0.03448276  0.03448276  0.10344828  0.17241379
  0.24137931  0.31034483  0.37931034  0.44827586  0.51724138  0.5862069
  0.65517241  0.72413793  0.79310345  0.86206897  0.93103448  1.        ]
[[inf inf inf inf inf inf inf inf inf inf  0.]
 [inf inf inf inf inf inf inf inf inf inf  0.]
 [inf inf inf inf inf inf inf inf inf inf  0.]
 [inf inf inf inf inf inf inf inf inf inf  0.]
 [inf inf inf inf inf inf inf inf inf inf  0.]
 [inf inf inf inf inf inf inf 